In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model

In [3]:

def create_dataset():
    # clone repo
    !git clone https://github.com/microsoft/FERPlus.git
    !mkdir ./FER2013plus
    # Extract the downloaded data into another directory
    !tar xvzf "../input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz" -C "./FER2013plus/"
    # Run the data generation script on the extracted data
    !python FERPlus/src/generate_training_data.py -d ./FERPlus/data -fer "FER2013plus/fer2013/fer2013.csv" -ferplus "./FERPlus/fer2013new.csv"
create_dataset()

In [4]:
col_names = ["usage","img_name", "neutral","happiness","surprise","sadness","anger","disgust","fear","contempt","Unknown", "NF"]
df = pd.read_csv('./FERPlus/fer2013new.csv',names=col_names,header=None,skiprows = 1)
print(df.shape)

In [5]:
df_copy = df.copy()
useful_cols = [x for x in df_copy.columns][2:]
dfc = df_copy[useful_cols]
dfc['emotion'] = dfc.idxmax(axis=1)
df_final = pd.concat([df_copy[['usage','img_name']],  dfc[['emotion']]], axis=1, ignore_index=False)
df_final['img_name'] = df_final['img_name'].apply(str)
df_final

In [6]:
count = 0 
for i in range(0,35886):
    if df_final['img_name'][i]=='nan':
        count+=1
print(count)
# df_final['img_name'].isna().sum()

In [7]:
# Preview image
img = cv2.imread('./FERPlus/data/FER2013Train/fer0000000.png')
plt.imshow(img)
print(img.shape)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255,
                             shear_range=0.2,
                             horizontal_flip=True,
                             brightness_range=(.6, 1.2),
                              zoom_range=0.2,
                              rotation_range = 30,
                              fill_mode='nearest')

train_generator=datagen.flow_from_dataframe(
    dataframe=df_final,
    directory="./FERPlus/data/FER2013Train",
    x_col="img_name",
    y_col="emotion",
    batch_size=64,
    class_mode="categorical",
    target_size=(224,224))

valid_datagen = ImageDataGenerator(rescale=1/255)
valid_generator=valid_datagen.flow_from_dataframe(
    dataframe=df_final,
    directory="./FERPlus/data/FER2013Valid",
    x_col="img_name",
    y_col="emotion",
    batch_size=64,
    class_mode="categorical",
    target_size=(224,224))

test_datagen = ImageDataGenerator(rescale=1/255)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=df_final,
    directory="./FERPlus/data/FER2013Test",
    x_col="img_name",
    y_col="emotion",
    batch_size=64,
    class_mode="categorical",
    target_size=(224,224))


In [9]:
df_final.emotion.value_counts().sort_index(ascending=False).plot(kind='bar')

In [10]:
import tensorflow as tf
import tensorflow_addons as tfa


## Transfer learning

In [11]:

inputs = Input(shape=(224, 224, 3))
model = EfficientNetB0(weights='imagenet', include_top=False, drop_connect_rate=0.4, input_tensor=inputs)

In [13]:
NUM_CLASSES = 10
IMG_SIZE = 224

In [14]:
from tensorflow.keras.applications import EfficientNetB0
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

outputs = EfficientNetB0(include_top=True, weights=None, drop_connect_rate=0.4, classes=NUM_CLASSES)(inputs)
#drop_connect_rate=0.4,
model = tf.keras.Model(inputs, outputs)


In [15]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size


In [16]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

history = model.fit(x = train_generator,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_generator,
          validation_steps = STEP_SIZE_VALID,
          epochs = 50)

In [17]:
history = history.history
n_epochs = len(history['loss'])

plt.figure(figsize=[14,4])
plt.subplot(1,2,1) m
plt.plot(range(1, n_epochs+1), history['loss'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), history['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.show()

In [18]:
model.evaluate(test_generator)

## Save and quantize model

In [19]:
# save and quantize model
model.save('./emotion.h5')
model = tf.keras.models.load_model('./emotion.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("emotion.tflite", "wb").write(tflite_model)